# Hello


In [ ]:
import numpy as np
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()
X=data.data
y=data.target.reshape(-1,1)

def sigmoid(x):
    return 1.0 / (1 + np.exp(-x))

def logistic_vanilla(X, y, num_iterations=10000, learning_rate=1e-2):
    r, c = X.shape
    X = np.hstack((np.ones((r, 1)), X))
    beta = 2 * np.random.randn(c + 1, 1) - 1

    for i in range(num_iterations):
        predictions = sigmoid(X.dot(beta))
        gradient = X.T.dot(y - predictions)
        beta += learning_rate * gradient

    return beta

def logistic_polyak(X, y, num_iterations=10000, learning_rate=1e-2, momentum=0.9):
    r, c = X.shape
    X = np.hstack((np.ones((r, 1)), X))
    beta = 2 * np.random.randn(c + 1, 1) - 1
    velocity = np.zeros_like(beta)

    for i in range(num_iterations):
        predictions = sigmoid(X.dot(beta))
        gradient = X.T.dot(y - predictions)
        velocity = momentum * velocity + learning_rate * gradient
        beta += velocity

    return beta

def logistic_nesterov(X, y, num_iterations=10000, learning_rate=1e-2, momentum=0.9):
    r, c = X.shape
    X = np.hstack((np.ones((r, 1)), X))
    beta = 2 * np.random.randn(c + 1, 1) - 1
    velocity = np.zeros_like(beta)

    for i in range(num_iterations):
        # Lookahead position
        lookahead_beta = beta + momentum * velocity
        predictions = sigmoid(X.dot(lookahead_beta))
        gradient = X.T.dot(y - predictions)
        velocity = momentum * velocity + learning_rate * gradient
        beta += velocity

    return beta

# Generate synthetic dataset
n, p = 1000, 5
X = np.random.normal(0, 1, (n, p))
beta_true = np.ones((p, 1))
Y = (np.random.uniform(0, 1, (n, 1)) < sigmoid(X.dot(beta_true))).astype(float)

# Run logistic regression using the different gradient descent methods
beta_vanilla = logistic_vanilla(X, Y)
beta_polyak = logistic_polyak(X, Y)
beta_nesterov = logistic_nesterov(X, Y)

print("Vanilla Gradient Descent Beta:", beta_vanilla.ravel())
print("Polyak Momentum Beta:", beta_polyak.ravel())
print("Nesterov Accelerated Gradient Beta:", beta_nesterov.ravel())



ValueError: operands could not be broadcast together with shapes (80,) (20,) 